In [1]:
## Receive the signal
include("rxsignal_withchannelandfreqoff.jl");

In [2]:
## Load the template signal
include("pss2.jl");

In [3]:
using FFTW

In [4]:
####################################################
# Add convenient Math functions and function aliases
####################################################

Fᴵ = ifft # Inverse Fourier Transform
# Convolution infix function
function ⊗(a, b) 
    return conv(a,b)
end
∑ = sum # Sum alias
argmax = findmax # Alias used for MLE search 

findmax (generic function with 8 methods)

In [5]:
## Assign the received signal a variable Ŝᵣₓ³
𝑅ₚₛₛ³ = rxs3; # RX Received Signal 3 File Handle
@show size(𝑅ₚₛₛ³); 

size(𝑅ₚₛₛ³) = (616447, 1)


In [6]:
using LinearAlgebra

In [7]:
## Assign the template signal a variable 𝐻ₚₛₛ²
## convert the signal in time domain
Pₚₛₛ² = pss_2; # File Handle
𝐻ₚₛₛ²ᵀ = transpose.(Pₚₛₛ²); # Hessian Transpose ?
𝐻ₚₛₛ² = Fᴵ(𝐻ₚₛₛ²ᵀ); # S Slanted (fourier transform) in time domain
𝐻ₚₛₛ² ./= norm(𝐻ₚₛₛ²);
𝐻ₚₛₛ² = vcat(𝐻ₚₛₛ²[(end-143):end], 𝐻ₚₛₛ²); @show size(𝐻ₚₛₛ²); # end is 2048 in this case, concat math 

size(𝐻ₚₛₛ²) = (2192,)


In [8]:
## Prepare the template signal for convolution
H̅ₚₛₛ² = reverse(𝐻ₚₛₛ²); # reverse
H̅ₚₛₛ²ᴴ = conj(H̅ₚₛₛ²); # conjugate 

In [9]:
using DSP

In [10]:
## Perform the convolution between the 2 signals
𝐻̂ₚₛₛ² = 10 * log10.(abs.( 𝑅ₚₛₛ³ ⊗ H̅ₚₛₛ²ᴴ )); @show size( 𝐻̂ₚₛₛ² );

size(𝐻̂ₚₛₛ²) = (618638, 1)


In [11]:
# Find maximum value and its index
Ĉᵩ², 𝑁̂𝑓² = argmax(𝐻̂ₚₛₛ²); @show Ĉᵩ², 𝑁̂𝑓²;

(Ĉᵩ², 𝑁̂𝑓²) = (50.649042476081405, CartesianIndex(6628, 1))


In [12]:
θ₁ = 144
θ₂ = 2048
getindex(𝑁̂𝑓², 1) - ( θ₂ + θ₁ )

4436

In [13]:
using Plots

In [14]:
# Plot the result
m2_chan_plot = plot(𝐻̂ₚₛₛ², xlabel="Sample", ylabel="Power (dB)", title="Convolution Result", ylim=(-20, 60))
savefig(m2_chan_plot,"images/m2_chan_plot.png");

<img src=images/m2_chan_plot.png width='' heigth='' > </img>

In [20]:
########################################################
# Frequency Offset Estimator Function
########################################################
function freq_offset_est(𝑅ₚₛₛ, 𝐻ₚₛₛ, Nf, m, 𝑓ₛ)

    # Frequency offset estimator
    Y = zeros(ComplexF64, length(m))
    L = length(𝐻ₚₛₛ); @show L
    t = 0:(1/𝑓ₛ):((L-1)/𝑓ₛ); @show t

    signal_part = 𝑅ₚₛₛ[Nf:(Nf + L -1)]; @show size( signal_part )
    transConj = conj(𝐻ₚₛₛ) .* transpose(signal_part); @show size( transConj )
    # transConj = conj(𝐻ₚₛₛ) .* signal_part'; @show size( transConj )

    for j = 1:length(m)
        exp_comp = exp.(-2*pi*im*m[j].*t)
        signal_offset = ∑(exp_comp .* transConj)
        value = abs2.(signal_offset)
        Y[j] += value
    end

    return Y
end

freq_offset_est (generic function with 1 method)

In [21]:
Δ𝑓 = 10.0

𝑓ₛ = 61.44e6
f_min = -7500.
f_max = 7500.

m = f_min:Δ𝑓:f_max; @show m
Nf = getindex(𝑁̂𝑓², 1) - length(𝐻ₚₛₛ²) + 1; @show Nf;

m = -7500.0:10.0:7500.0
Nf = 4437


In [22]:
# Grab the function Profs Frequency Offset with his values
Y = freq_offset_est(𝑅ₚₛₛ³, 𝐻ₚₛₛ², Nf, m, 𝑓ₛ);

L = 2192
t = 0.0:1.6276041666666667e-8:3.566080729166667e-5
size(signal_part) = (2192,)
size(transConj) = (2192, 2192)


In [23]:
Y_1 = 10 * log10.(abs.(Y))
cfo_estim_plot = 
plot(Y_1, xlabel="Sample", ylabel="Power (dB)", title="Estimation Result"
    # , xlim=(0, 1600)
    # , ylim=(1e10, 1.40e10)
)
savefig(cfo_estim_plot,"images/cfo_estim_plot.png");

<img src=images/cfo_estim_plot.png width='' height='' > </img>